# Statistical Modeling

We will go through the following process:
   ** Explore data -> Fit model -> Evaluate model -> Deploy model**

In [ ]:
# Import relevant libraries
import numpy as np 
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import missingno # To install this package with conda run one of the following: conda install -c conda-forge missingno 

# Step 1. Explore Data

Perform exploratory analysis on the variables using the whole data set.
Describe the data and comment on your observations/findings.

In [ ]:
#df = pd.read_excel('Glycohemoglobin_t1.xlsx')
#df = pd.read_excel('Glycohemoglobin_t3.xlsx')
#df = pd.read_excel('Glycohemoglobin_t4.xlsx')
df = pd.read_excel('Glycohemoglobin_t4_ShortName.xlsx')

In [ ]:
df.head()

In [ ]:
missingno.matrix(dftrain2, figsize = (30,10))

In [ ]:
# To check data types, type: df.dtypes
df.dtypes

In [ ]:
df.shape

In [ ]:
df.describe(include='all')

In [ ]:
# To visualise correlation matrix in a heatmap, type: sns.heatmap(df.corr(), annot=True, cmap='coolwarm');
# sns.heatmap(df.corr(), annot=True, cmap='coolwarm');
plt.figure(figsize=(12, 10))  # Adjust the figure size
heatmap = sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 8})
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=8)
plt.show()

In [ ]:
def correlation_heatmap(df):
    plt.figure(figsize=(16, 14))
    colormap = sns.diverging_palette(220, 10, as_cmap=True)
    
    sns.heatmap(
        df.corr(), 
        cmap=colormap,
        square=True, 
        cbar_kws={'shrink': 0.9}, 
        annot=True, 
        linewidths=0.9,
        vmax=1.0, 
        linecolor='white',
        annot_kws={'fontsize': 9}
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.show()

# Assuming 'df' is your DataFrame
correlation_heatmap(df)

In [ ]:
# To visualise pairwise relationship using Seaborn pairplot, type: sns.pairplot(df);
# Caution: It may take some time to generate a pairplot, especially when number of features is large.
sns.pairplot(df);

# Step 2. Fit Model

Split the data set into training set and testing set in a (approximate) ratio 75:25.
Set random state/seed using the last 4 digits of your SP admission number.
Fit the full additive MLR model on the training set.

#### Build a full MLR model for Y (pCASP9) using the predictors (X1 to X10, Genotype, Treatment and Behavior).

In [ ]:
# Split data set into 75:25.

from sklearn.model_selection import train_test_split
dftrain, dftest = train_test_split(df, test_size=0.25, random_state=1938)

# If you want to confirm ratio splitted:
print( len(dftrain)/len(df) )
print( len(dftest)/len(df) )

In [ ]:
#mlr_full = ols("diabetes ~ Sex + Age + Race_Or_Ethnicity + Income_Min +	Income_Max + On_Insulin_or_Diabetes_Meds + Weight_Kg + Height_cm + BMI + Upper_Leg_Length_cm + Upper_Arm_Length_cm + Arm_Circumference_cm + Waist_Circumference_cm + Triceps_Skinfold_mm + Subscapular_Skinfold_mm  + Albumin_g-dL + Blood_urea_nitrogen_mg-dl + Creatinine_mg-dl + gh#", dftrain).fit()
#mlr_full = ols(
#    "diabetes ~ Sex + Age + Race_Or_Ethnicity + Income_Min + Income_Max + On_Insulin_or_Diabetes_Meds + " 
#    "Weight_Kg + Height_cm + BMI + Upper_Leg_Length_cm + Upper_Arm_Length_cm + Arm_Circumference_cm + " 
#    "Waist_Circumference_cm + Triceps_Skinfold_mm + Subscapular_Skinfold_mm + Albumin_g_dL + " 
#    "Blood_urea_nitrogen_mg_dl + Creatinine_mg_dl + gh", dftrain
#).fit()

formula = (
    "diabetes ~ Sex + Age + Race_Or_Ethnicity + Income_Min + Income_Max + On_Insulin_or_Diabetes_Meds + Weight_Kg + Height_cm + BMI + Up_Leg_Len + Up_Arm_Len + Arm_Cir + Waist_Cir + Triceps_Sk + Subscapular_Sk + Alibumin + Blood_Urea + Creatinine + gh"
)
mlr_full = ols(formula, dftrain).fit()

print(mlr_full.summary())
print('\nMSE =', mlr_full.mse_resid)

# Step 3: Evaluate Model

Conduct relevant diagnostics on the full MLR model fitted.
Evaluate the model from the perspectives of model fit, prediction accuracy, model/predictor significance, and checking of assumptions.

How well does the model fit the data?
Is the model likely to be useful for prediction?
Are any of the basic assumptions violated?

We can perform the following dianostics:
- Check goodness of fit ($R^2$)
- Check accuracy (MSE)
- Test the slope and overall model
- Check assumptions by producing residual plots

## _Check goodness of fit ($R^2$) and accuracy (MSE)_

In [ ]:
# To extract Rsq value, type: modelname.rsquared
mlr_full.rsquared

### 60%, moderate fit, of the variability 

## _Check assumptions: normality_

We can use visualizations or run statistical tests to check if residuals satisfy the normality assumption.
- Histogram of residual should be bell-shaped.
- QQ plot of residuals should be a diagonal straight line.
- [Jarque-Bera test](https://en.wikipedia.org/wiki/Jarque%E2%80%93Bera_test): we want to see "large" p-value so as not to reject normality. [[Documentation]](https://www.statsmodels.org/dev/generated/statsmodels.stats.stattools.jarque_bera.html#statsmodels.stats.stattools.jarque_bera)
- Ominibus test: we want to see "large" p-value so as not to reject normality. See references in [statsmodel](https://www.statsmodels.org/dev/generated/statsmodels.stats.stattools.omni_normtest.html#statsmodels.stats.stattools.omni_normtest) or the [scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html).

In [ ]:
# To produce residual plots, we have to extract the residuals first. Type: residname = modelname.resid
res = mlr_full.resid
len(res)

In [ ]:
# Check whether residuals are normally distributed visually in a histogram.
# Type: plt.hist(residname);
plt.hist(res);

In [ ]:
# Alternatively, use a distribution plot which is a histogram with a smooth curve fitted to it.
# Type: sns.distplot(residname);
sns.distplot(res);

In [ ]:
# Construct a QQ plot of residuals. Type: sm.qqplot(residname, line='s');
sm.qqplot(res, line='s');

## To perform the Jarque-Bera test on residuals, type: sm.stats.jarque_bera(residname)
#### The function returns: JB test statistic, p-value, estimated skewness, estimated kurtosis


In [ ]:
sm.stats.jarque_bera(res)

## To perform the Omnibus test on residuals, type: sm.stats.omni_normtest(residname)

In [ ]:
sm.stats.omni_normtest(res)

## Conduct Shapiro-Wilk normality test. The function returns test statistics and P-value.

In [ ]:
from scipy.stats import shapiro
shapiro(res)

## Two ways to to conduct Anderson-Darling normality test
#### One uses the scipy library, and one uses the statsmodel library
#### scipy: returns test statistic and critical values. Reject H0 if test statistic > critical value
#### statsmodel: returns test statistics and P-value

In [ ]:
from scipy.stats import anderson
anderson(res)

In [ ]:
sm.stats.normal_ad(res)

## _Check Assumption: Homoscedasticity_

We can use visualization or run statistical test to check if residuals satisfy the homoscedasticity assumption.
- Plot of residuals vs. fitted values should be "bounded in a horizontal band".
- [Breusch–Pagan test](https://en.wikipedia.org/wiki/Breusch%E2%80%93Pagan_test): we want to see "large" p-value so as not to reject homoscedasticity. [[Documentation]](https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.het_breuschpagan.html#statsmodels.stats.diagnostic.het_breuschpagan)

In [ ]:
sns.residplot(x=mlr_full.fittedvalues, y=res, lowess=True)
plt.title('Residual Plot')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()

In [ ]:
# Perform the Breusch-Pagan test on residuals. Type: sm.stats.het_breuschpagan(residname, modelname.model.exog)
# The function returns: Lagrange multiplier statistic, p-value, F-value for B-P Constant Variance, F p-value
sm.stats.het_breuschpagan(res, mlr_full.model.exog)

#### Reject H0 (p-value = 1.1201)
#### Non-constant variance, even non-linear effect

## _Check assumptions: independence_

In [ ]:
# Plot a line chart of the residuals, according to their observed order, to visually check independence.
# Type: 
plt.plot(res);
# Add in titles and axes labels
plt.title('Residuals vs Observation Order')
plt.xlabel('Observation Order')
plt.ylabel('Residuals');

#### No autocorrelation

# Perform the Durbin-Watson test on residuals. Type: sm.stats.durbin_watson(residname)

In [ ]:
sm.stats.durbin_watson(res)
# no autocorrelation

## _Multicollinearity_

We can check for multicollinearity in the data set by:
- Visually inspect the correlation matrix for correlated predictors.
- Check the [condition number](https://en.wikipedia.org/wiki/Condition_number).  As a general rule of thumb, if the condition number is more than 30, the regression model may have multicolliearity problem. This is a matrix algebra method. In Statsmodel, summary output will automatically highlight multicollinearity issue if the condition number is too high.
- Check the [Variance Inflation Factor](https://en.wikipedia.org/wiki/Variance_inflation_factor) (VIF) method. As a rule of thumb, we interpret VIF for each predictor as follows:

| VIF | Interpretation |
| ----- | ----- |
| VIF = 1 | Not correlated |
| 1<VIF<5 | Moderately correlated |
| VIF > 5 | Highly correlated |

In [ ]:
# Construct a heatmap to check correlations between predictors.
plt.figure(figsize=(12, 10))  # Adjust the figure size
heatmap = sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 8})
heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
heatmap.set_yticklabels(heatmap.get_yticklabels(), fontsize=8)
plt.show()

No serious multicollinearity, continue to check

In [ ]:
# To extract condition number, type: modelname.condition_number
mlr_full.condition_number # for the whole model
#still acceptable, why so high? 

### Condition Number is high  (5068760.6)

# Check Multicollinearity (VIF)
# To print out all VIFs by a "for" loop:

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

# Determine the number of predictors
num_predictors = len(mlr_full.model.exog_names)

# Loop through each predictor to calculate VIF
for i in range(num_predictors):
    predictor = mlr_full.model.exog_names[i]
    VIF = vif(mlr_full.model.exog, i)
    print(predictor, VIF)

### For Sex, Age, Race and On_Insulin_or_Diabetes_Med, Up_Leg_Len, Up_Arm_Len are not correlated
### On_Insulin_or_Diabetes_Meds, Triceps_Sk, Subscapular_Sk, Alibumin, Blood_Urea, Creatinin and gh (Glycohemoglobin) are moderately correlated
### Income_Min  and Income_Max, Weight_Kg, Height_cm, BMI, Arm_Cir, Waist_Cir,  are highly correlated

# Step 4: Improve Model
#Improve the model using at least 4 of the following techniques where appropriate:
* Removing outlier(s) (if any)
* Centering and/or standardizing of variables
* Principal component analysis (PCA)
* Transformation of variables
* Interaction of variables
* Variable selection

Explain how the model is improved after applying each of the techniques.
* [Remark: There is no “best” model or “standard” solution.]

# 1. Removing Outlier(s) (if any) from the residuals

In [ ]:
res.idxmin()  ## or, res.idxmax() 

In [ ]:
res[229]

In [ ]:
df.iloc[229,:]

In [ ]:
df2 = df.drop([229], axis=0)
df2.shape

In [ ]:
# Split data set into 75:25.

from sklearn.model_selection import train_test_split
dftrain2, dftest2 = train_test_split(df2, test_size=0.25, random_state=1938)

# If you want to confirm ratio splitted:
print( len(dftrain2)/len(df2) )
print( len(dftest2)/len(df2) )

In [ ]:
print(dftrain2.isna().any())

In [ ]:
missingno.matrix(dftrain2, figsize = (30,10))

In [ ]:
formula = (
    "diabetes ~ Sex + Age + On_Insulin_or_Diabetes_Meds + Height_cm + BMI + Arm_Cir + Waist_Cir + Alibumin + Blood_Urea + Creatinine + gh"
)
mlr_full_2 = ols(formula, dftrain).fit()

print(mlr_full_2.summary())
print('\nMSE =', mlr_full_2.mse_resid)

### The 2nd model has slightly improved wiht Adj. R-sqr 0.613.
### However, MSE is about the same.

In [ ]:
# Perform Backward Selection based on p-Values. Removed predictor with p-Values higher than 0.05 

mlr_full_3 = ols(  "diabetes ~ Sex + Age + On_Insulin_or_Diabetes_Meds + Height_cm + BMI + Arm_Cir + Waist_Cir + Alibumin + Blood_Urea + Creatinine + gh", dftrain2).fit()
print(mlr_full_3.summary())
print('\nMSE =', mlr_full_3.mse_resid)

In [ ]:
# Perform Backward Selection based on p-Values. Removed predictor with p-Values higher than 0.05 

mlr_full_3 = ols(  "diabetes ~ Age + On_Insulin_or_Diabetes_Meds + BMI + Height_cm +  Creatinine + gh", dftrain2).fit()
print(mlr_full_3.summary())
print('\nMSE =', mlr_full_3.mse_resid)

### The MSE of the models are slightly getting lower after removing the predictors (p-value > 0.05).
### Adj. R2 has slightly improved. (62%)


In [ ]:
res3 = mlr_full_3.resid
len(res3)

In [ ]:
sns.residplot(x=mlr_full_3.fittedvalues, y=res3)
plt.title('Residual Plot')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()
#non linear pattern

In [ ]:
### The residual plot is about the same as above.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

for i in range(6):
    predictor = mlr_full_3.model.exog_names[i]
    Vif3 = vif(mlr_full_3.model.exog,i)
    print(predictor,Vif3)

### The multicollinearity problem is resolved or improved. It becomes moderately correlated for individual predictors (1<VIF<5).

# 3. Create interaction of variables

In [ ]:
# Create interation of variables
#"diabetes ~ Age + On_Insulin_or_Diabetes_Meds + BMI + Height_cm +  Creatinine + gh"
mlr_full_4 = ols("diabetes ~ Age + On_Insulin_or_Diabetes_Meds + BMI + Height_cm +  Creatinine + gh + BMI*Creatinine + On_Insulin_or_Diabetes_Meds*gh", dftrain2).fit()
print(mlr_full_4.summary())
print('\nMSE =', mlr_full_4.mse_resid)

### Adj. R2 has been improved. (62.4%)
### The MSE of this model is getting lower. (0.0432)
### Cond. No. is 4.16e+03.

# 4. Transform variables

### Removed some of the predictors with p-value > 0.05

In [ ]:
mlr_full_5 = ols("diabetes ~ Age + I(On_Insulin_or_Diabetes_Meds**2) + BMI +  Creatinine + gh  + On_Insulin_or_Diabetes_Meds*gh", dftrain2).fit()
print(mlr_full_5.summary())
print('\nMSE =', mlr_full_5.mse_resid)

In [ ]:
res5 = mlr_full_5.resid
len(res5)

In [ ]:
sns.residplot(x=mlr_full_5.fittedvalues, y=res5)
plt.title('Residual Plot')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.show()
#linear pattern

In [ ]:
# To inspect skewness, produce histograms
fig = plt.figure(figsize=(12,4))

ax = fig.add_subplot(121)
plt.hist(dftrain2.diabetes)  # the response Y
plt.title('histogram of response');

ax = fig.add_subplot(122)
plt.hist(res5)  # the residuals
plt.title('histogram of residual');

## Both plot are right skewiness